# Shift
Move CH4 and C2H6 to the binding site where the ligand was

In [29]:
from pathlib import Path
import gzip

import numpy as np

from openmm import app, openmm, unit
from grandfep import utils


In [9]:
pdb_a   = app.PDBFile("../lig0/06_solv.pdb")
pdb_b   = app.PDBFile("../lig1/06_solv.pdb")
pdb_lig = app.PDBFile("02-lig.pdb")

In [7]:
pos_a = pdb_a.getPositions(asNumpy=True)[:5]

Quantity(value=array([[1.5171, 1.5285, 1.4657],
       [1.5528, 1.4277, 1.4657],
       [1.5528, 1.579 , 1.5531],
       [1.5528, 1.579 , 1.3783],
       [1.4101, 1.5286, 1.4657]]), unit=nanometer)

In [8]:
pos_b = pdb_b.getPositions(asNumpy=True)[:8]

Quantity(value=array([[1.5171, 1.5285, 1.4657],
       [1.4578, 1.4445, 1.4948],
       [1.6005, 1.4949, 1.4075],
       [1.4578, 1.5958, 1.4074],
       [1.5684, 1.6011, 1.5914],
       [1.4852, 1.6346, 1.6497],
       [1.628 , 1.5339, 1.6496],
       [1.6279, 1.6852, 1.5623]]), unit=nanometer)

In [13]:
pos_0 = pdb_lig.getPositions(asNumpy=True)[0]
pos_0

Quantity(value=array([ 0.1925, -0.2459, -0.1726]), unit=nanometer)

In [17]:
pdb = pdb_a
pos = pdb.getPositions(asNumpy=True)
pos_shifted = pos + (pos_0 - pos[0])
with open("03-A-CH4.pdb", "w") as f:
    app.PDBFile.writeFile(pdb.topology, pos_shifted, f)

In [18]:
pdb = pdb_b
pos = pdb.getPositions(asNumpy=True)
pos_shifted = pos + (pos_0 - pos[0])
with open("03-B-C2H6.pdb", "w") as f:
    app.PDBFile.writeFile(pdb.topology, pos_shifted, f)

# Hybrid System

In [34]:
nonbonded_Amber = {"nonbondedMethod": app.PME,
                   "nonbondedCutoff": 1.0 * unit.nanometer,
                   "constraints": app.HBonds,
                   }

In [45]:
inpcrdA = app.AmberInpcrdFile("05-stateA.rst7")
inpcrdB = app.AmberInpcrdFile("05-stateB.rst7")
prmtopA = app.AmberPrmtopFile("05-stateA.prmtop", periodicBoxVectors=inpcrdA.boxVectors)
prmtopB = app.AmberPrmtopFile("05-stateB.prmtop", periodicBoxVectors=inpcrdB.boxVectors)
nonbonded = nonbonded_Amber

sysA = prmtopA.createSystem(**nonbonded)
sysB = prmtopB.createSystem(**nonbonded)

In [46]:
mapping_list = [{'res_nameA': 'MOL',
                 'res_nameB': 'MOL',
                 'index_map': {0: 0}}]
old_to_new_atom_map, old_to_new_core_atom_map = utils.prepare_atom_map(prmtopA.topology, prmtopB.topology, mapping_list)
h_factory = utils.HybridTopologyFactory(
    sysA, inpcrdA.getPositions(), prmtopA.topology, sysB, inpcrdB.getPositions(), prmtopB.topology,
    old_to_new_atom_map,      # All atoms that should map from A to B
    old_to_new_core_atom_map, # Alchemical Atoms that should map from A to B
    use_dispersion_correction=True,
    softcore_LJ_v2=False)

In [47]:
system    = h_factory.hybrid_system
topology  = h_factory.omm_hybrid_topology
positions = h_factory.hybrid_positions

In [48]:
with gzip.open("system_hybrid.xml.gz", mode="wt") as fh:
    fh.write(openmm.XmlSerializer.serialize(system))

with open("system_hybrid.pdb", "w") as f:
    app.PDBFile.writeFile(topology, positions, f)

In [49]:
len([at for at in prmtopA.topology.atoms()])

3532

In [50]:
len([at for at in prmtopB.topology.atoms()])

3535

In [51]:
len([at for at in topology.atoms()])

3539